In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Read in data
products = pd.read_csv("Resources/products.csv")
accounts = pd.read_csv("Resources/accounts.csv")
ast = pd.read_csv("Resources/account_state_transitions.csv")
gpa = pd.read_csv("Resources/group_product_associations.csv")
groups = pd.read_csv("Resources/groups.csv")
payments = pd.read_csv("Resources/payments.csv")

## 1) Generate a bar chart visualizing the relative frequency of Products registered with Accounts.

In [3]:
gpa = gpa.rename(columns={"id":"gpa_id"})
accounts = accounts.rename(columns={"id":"accounts_df_id"})

In [4]:
# Merge accounts and gpa data frames
group_id_merge = gpa.merge(accounts, how="inner",on="group_id")
group_id_merge.head()

,gpa_id,group_id,product_id,accounts_df_id,organization_id,registration_date,is_written_off,is_unlocked
0,1,1,42,2,10001,2019-11-06 05:49:39.571392,False,True
1,1,1,42,3,10001,2019-11-14 14:57:02.571392,False,False
2,1,1,42,4,10001,2019-12-11 03:18:53.571392,False,True
3,1,1,42,5,10001,2019-10-31 22:36:23.571392,False,False
4,1,1,42,6,10001,2019-09-29 13:04:45.571392,False,True


In [5]:
# Check w/ group by and manually
df_product_registration = pd.DataFrame(group_id_merge['product_id'].value_counts())
df_product_registration = df_product_registration.reset_index().sort_values('index')
df_product_registration = df_product_registration.rename(columns={"index":"product_id","product_id":"registration_count"})
df_product_registration = df_product_registration.reset_index(drop=True)
df_product_registration.head()

,product_id,registration_count
0,1,21
1,2,11
2,3,11
3,4,21
4,5,11


In [119]:
# Make Bar Chart -- tuples?

## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

## 3) Devise a method of generating a table of all Accounts with their cumulative payments to date as of any arbitrary timestamp in the past.

add timestamp portion

In [23]:
# Variable to take timestamp input.
timestamp = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2019-12-26


In [24]:
print(timestamp)

2019-12-26


In [29]:
payments.head()

,id,effective_when,account_id,currency,amount
0,1,2019-12-26 06:00:42.571392,1,KES,950
1,2,2019-12-26 06:00:42.571392,1,KES,950
2,3,2019-11-06 05:49:39.571392,2,KES,950
3,4,2019-11-06 05:49:39.571392,2,KES,950
4,5,2019-11-11 06:43:34.571392,2,KES,415


In [46]:
# Select rows containing indicated timestamp
a = payments[payments['effective_when'].str.contains(timestamp,case=False)]
a

,id,effective_when,account_id,currency,amount
0,1,2019-12-26 06:00:42.571392,1,KES,950
1,2,2019-12-26 06:00:42.571392,1,KES,950
175,176,2019-12-26 03:10:17.571392,12,KES,140
239,240,2019-12-26 21:20:35.571392,14,KES,97
302,303,2019-12-26 03:23:31.571392,17,KES,88
325,326,2019-12-26 09:16:43.571392,18,KES,136
860,861,2019-12-26 16:31:27.571392,48,KES,399
980,981,2019-12-26 01:15:23.571392,58,KES,279
995,996,2019-12-26 16:17:48.571392,59,KES,242
1099,1100,2019-12-26 14:27:40.571392,62,KES,118


In [44]:
# SELECT all rows WHERE effect_when[i][:10] == timestamp
emptylist = []

for item in payments['effective_when']: 
    date = item[:10]
    if date == timestamp:
        emptylist.append(date)
#         print(payments['account_id'])

In [45]:
len(emptylist)

69

In [7]:
# Set currency
currency = payments.currency[0]

In [8]:
# Group by account_id and count number of records
num_payments = payments.groupby('account_id').count()

# Flatten dataframe
num_payments = num_payments.rename_axis(None, axis=1).reset_index()

# Drop columns
num_payments = num_payments.drop(columns=['effective_when','currency','amount'])

# Rename column
cumulative_payments = num_payments.rename(columns={"id":"total_num_payments"})
cumulative_payments.head()

,account_id,total_num_payments
0,1,2
1,2,18
2,3,16
3,4,13
4,5,14


In [9]:
# Group by account_id and sum payment amounts
sum_payments = payments.groupby('account_id').sum()

# Sort values by id
sum_payments = sum_payments.sort_values('id')

# Flatten dataframe
sum_payments = sum_payments.rename_axis(None, axis=1).reset_index()

# Select one column
sum_payments = sum_payments['amount']

# Add series to cumulative_payments dataframe
cumulative_payments[f'total_sum_payments_{currency}'] = sum_payments
cumulative_payments.head()

,account_id,total_num_payments,total_sum_payments_KES
0,1,2,1900
1,2,18,8100
2,3,16,5217
3,4,13,7661
4,5,14,8100


## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.